In [2]:
pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [2]:
import networkx as nx
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import numpy as np
from networkx.readwrite import json_graph
import json
from sklearn.manifold import TSNE
import plotly.express as px


from node2vec import Node2Vec

In [3]:
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/AnasAito/Semantic-parser/main/topicgraph/UM6P%20final%20graph.json

--2023-08-20 04:58:10--  https://raw.githubusercontent.com/AnasAito/Semantic-parser/main/topicgraph/UM6P%20final%20graph.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375813 (367K) [text/plain]
Saving to: ‘data/UM6P final graph.json’

UM6P final graph.js 100%[===================>] 367.00K  --.-KB/s    in 0.04s   

2023-08-20 04:58:11 (8.55 MB/s) - ‘data/UM6P final graph.json’ saved [375813/375813]



In [8]:
with open('data/UM6P final graph.json') as json_file:
    data = json.load(json_file)
    graph = nx.node_link_graph(data)

In [9]:
def get_similarity(graph,model,topic,topics):

    conn=[]

    for node in graph.nodes :
        if node in topics:
            pass

        else :

            try :
                w =  model.wv.distance(topic,node)
                conn.append(w)
                ##conn_index.append(node)
            except :
                conn.append(1000)
    return conn

def get_df(graph,model,topics):
    dict_values={}
   # topics = ['plant','waste','battery','soil']
    for topic in topics :
        print('-------------')
        print(topic)
        value_list = get_similarity(graph,model,topic,topics)
        ##normed = (np.array(value_list) - min(value_list))/(max(value_list)-min(value_list))
        dict_values[topic] = value_list
    filtred = [node for node in graph.nodes if node not in topics]
    df = pd.DataFrame(data=dict_values).set_index([pd.Index(filtred)])
    return df





def add_communities(graph,df,topics):
    dict_ = {}
    for topic in topics :
        dict_[topic] =[]


    for node in graph.nodes:
        if node in topics:
            group = -1
        else :
            row = df.loc[ node , : ]
            values = list(row)
            min_index = values.index(min(values))
            dict_[topics[min_index]].append(node)
            group = min_index
        graph.nodes[node]['group']=group
    return dict_

In [10]:
topics = [
 'cellulose',
 'sludge',
 'plant',
 'waste',
 'phosphate',
 'material',
 'soil']


## model for distace computation
node2vec = Node2Vec(graph,q=0.25, dimensions=20, walk_length=16, num_walks=100, workers=4)  # Use temp_folder for big graphs
model = node2vec.fit(window=10, min_count=1, batch_words=4)


df = get_df(graph,model,topics)
clusters = add_communities(graph,df,topics)
#data = json_graph.node_link_data(graph)




# Writing a JSON file
#with open('./data.json', 'w') as f:
 #   json.dump(data, f)

Computing transition probabilities:   0%|          | 0/1145 [00:00<?, ?it/s]

-------------
cellulose
-------------
sludge
-------------
plant
-------------
waste
-------------
phosphate
-------------
material
-------------
soil


In [11]:
df.loc['sugarcane']

cellulose    0.257833
sludge       0.787018
plant        0.782887
waste        0.322227
phosphate    0.866103
material     0.845783
soil         0.882163
Name: sugarcane, dtype: float64

In [12]:
clusters['cellulose']

['mechanical',
 'film',
 'blend',
 'bagasse',
 'property',
 'characterization',
 'polymer',
 'sugarcane',
 'nanocrystal',
 'crystalline',
 'hydrolysi',
 'acid',
 'fiber',
 'biopolymer',
 'nanocomposite',
 'stipa',
 'tenacissima',
 'tensile',
 'graphene',
 'oxide',
 'nanofiller',
 'synergistic',
 'hybrid',
 'reinforcement',
 'extraction',
 'food',
 'package',
 'kcarrageenan',
 'halpintsai',
 'p2type',
 'miscanthu',
 'suspension',
 'aqueous',
 'group',
 'rheologic',
 'sulfate',
 'biological',
 'shell',
 'dye',
 'adsorbent',
 'cocoa',
 'sunflower',
 'oil',
 'cake',
 'composite',
 'coating',
 'lignin',
 'polyethylene',
 'mapk',
 'screening',
 'p38',
 'discovery',
 'molecular',
 'silico',
 'inhibit',
 'dock',
 'physical',
 'glycerol',
 'plastic',
 'bionanocomposite',
 'microfiber',
 'physicochemic',
 'nanofibril',
 'polysaccharide',
 'neck',
 'head',
 'squamous',
 'elrcxcl',
 'carcinoma',
 'sunitinib',
 'cxcr12',
 'photolatent',
 'catalyst',
 'heterogeneous',
 'rufinamide',
 'azidealkyne',


In [13]:
model.wv.distance('material','battery')

0.2097063660621643